In [1]:
import glob
import nltk
import pandas as pd

In [2]:
data = pd.read_csv('../fulldata/sites.csv')

In [3]:
data

,filename,websites,titles
0,theatlantic.txt,theatlantic.com,The Atlantic
1,imdb.txt,imdb.com,IMDB
2,nytimes.txt,nytimes.com,The New York Times
3,voxmedia.txt,voxmedia.com,Vox
4,nbcuniversal.txt,nbcuniversal.com,NBC Universal Media
...,...,...,...
108,dailynews.txt,dailynews.com,Los Angeles Daily News
109,lids.txt,lids.com,Lids
110,sports-reference.txt,sports-reference.com,Sports Reference
111,foxsports.txt,foxsports.com,Fox Sports Insider


In [4]:
filepath = '../fulldata/textpolicies/'
def readtxt(fn):
    f = open(glob.glob(filepath + fn)[0])
    text = f.read()
    f.close()
    return text

data['content'] = data['filename'].apply(readtxt)

data.head()

,filename,websites,titles,content
0,theatlantic.txt,theatlantic.com,The Atlantic,"*Privacy Policy *\n\n*Effective: January 1, 20..."
1,imdb.txt,imdb.com,IMDB,"IMDb Privacy Notice\n\n|||Last Updated, Decemb..."
2,nytimes.txt,nytimes.com,The New York Times,"*Privacy Policy *\n\nLast Updated on June 10, ..."
3,voxmedia.txt,voxmedia.com,Vox,Vox Media Privacy Policy\n\n|||*Updated as of ...
4,nbcuniversal.txt,nbcuniversal.com,NBC Universal Media,Full Privacy Policy\n\nLast updated: 14 Januar...


In [5]:
len(data)

113

In [6]:
for i in range(len(data)):
    data.content[i] = data.content[i].split('|||')
    tmplist = data.content[i]
    tmpstring = ' '.join(tmplist)
    data.content[i] = tmpstring
    
    data.content[i] = data.content[i].split('*')
    tmplist = data.content[i]
    tmpstring = ' '.join(tmplist)
    data.content[i] = tmpstring
    
    data.content[i] = data.content[i].split('\n')
    tmplist = data.content[i]
    tmpstring = ' '.join(tmplist)
    data.content[i] = tmpstring
    
    data.content[i] = data.content[i].split(':')
    tmplist = data.content[i]
    tmpstring = ' '.join(tmplist)
    data.content[i] = tmpstring
    
    data.content[i] = data.content[i].split(' ')

In [7]:
data

,filename,websites,titles,content
0,theatlantic.txt,theatlantic.com,The Atlantic,"[, Privacy, Policy, , , , , Effective, , Janua..."
1,imdb.txt,imdb.com,IMDB,"[IMDb, Privacy, Notice, , , Last, Updated,, De..."
2,nytimes.txt,nytimes.com,The New York Times,"[, Privacy, Policy, , , , Last, Updated, on, J..."
3,voxmedia.txt,voxmedia.com,Vox,"[Vox, Media, Privacy, Policy, , , , Updated, a..."
4,nbcuniversal.txt,nbcuniversal.com,NBC Universal Media,"[Full, Privacy, Policy, , Last, updated, , 14,..."
...,...,...,...,...
108,dailynews.txt,dailynews.com,Los Angeles Daily News,"[, PRIVACY, POLICY, , , , This, policy, descri..."
109,lids.txt,lids.com,Lids,"[Privacy, Policy, , , Last, updated, , August,..."
110,sports-reference.txt,sports-reference.com,Sports Reference,"[SPORTS, REFERENCE, LLC, -, Privacy, Statement..."
111,foxsports.txt,foxsports.com,Fox Sports Insider,"[Privacy, Policy, Effective, Date, , June, 11,..."


In [8]:
for i in range(len(data)):
    ct = 0
    while ct < (len(data.content[i])):
        if data.content[i][ct] == '' or data.content[i][ct] == ' ':
            del data.content[i][ct]
        else:
            ct += 1

In [9]:
data

,filename,websites,titles,content
0,theatlantic.txt,theatlantic.com,The Atlantic,"[Privacy, Policy, Effective, January, 1,, 2015..."
1,imdb.txt,imdb.com,IMDB,"[IMDb, Privacy, Notice, Last, Updated,, Decemb..."
2,nytimes.txt,nytimes.com,The New York Times,"[Privacy, Policy, Last, Updated, on, June, 10,..."
3,voxmedia.txt,voxmedia.com,Vox,"[Vox, Media, Privacy, Policy, Updated, as, of,..."
4,nbcuniversal.txt,nbcuniversal.com,NBC Universal Media,"[Full, Privacy, Policy, Last, updated, 14, Jan..."
...,...,...,...,...
108,dailynews.txt,dailynews.com,Los Angeles Daily News,"[PRIVACY, POLICY, This, policy, describes, the..."
109,lids.txt,lids.com,Lids,"[Privacy, Policy, Last, updated, August, 26,, ..."
110,sports-reference.txt,sports-reference.com,Sports Reference,"[SPORTS, REFERENCE, LLC, -, Privacy, Statement..."
111,foxsports.txt,foxsports.com,Fox Sports Insider,"[Privacy, Policy, Effective, Date, June, 11,, ..."


In [10]:
data['length'] = None
for i in range(len(data)):
    data['length'][i] = len(data['content'][i])

In [11]:
data

,filename,websites,titles,content,length
0,theatlantic.txt,theatlantic.com,The Atlantic,"[Privacy, Policy, Effective, January, 1,, 2015...",3323
1,imdb.txt,imdb.com,IMDB,"[IMDb, Privacy, Notice, Last, Updated,, Decemb...",2033
2,nytimes.txt,nytimes.com,The New York Times,"[Privacy, Policy, Last, Updated, on, June, 10,...",3771
3,voxmedia.txt,voxmedia.com,Vox,"[Vox, Media, Privacy, Policy, Updated, as, of,...",2441
4,nbcuniversal.txt,nbcuniversal.com,NBC Universal Media,"[Full, Privacy, Policy, Last, updated, 14, Jan...",4174
...,...,...,...,...,...
108,dailynews.txt,dailynews.com,Los Angeles Daily News,"[PRIVACY, POLICY, This, policy, describes, the...",3388
109,lids.txt,lids.com,Lids,"[Privacy, Policy, Last, updated, August, 26,, ...",2359
110,sports-reference.txt,sports-reference.com,Sports Reference,"[SPORTS, REFERENCE, LLC, -, Privacy, Statement...",1713
111,foxsports.txt,foxsports.com,Fox Sports Insider,"[Privacy, Policy, Effective, Date, June, 11,, ...",3122


In [12]:
data['length'].max()

7772

In [13]:
data['length'].min()

81